# GymPy

Note:
Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

In [2]:
# Part 1 data file
master_data_path = "Output/master_data.csv"
lat_lng_path1 = "Resources/australian_postcodes_loc.csv"
lat_lng_path2 = "Output/postcode_loc_df.csv"

# Read the metadata 
master_data_df = pd.read_csv(master_data_path)
lat_lng_df1 = pd.read_csv(lat_lng_path1)
#lat_lng_df2 = pd.read_csv(lat_lng_path2)

In [3]:
master_data_df.shape

(2641, 21)

In [4]:
lat_lng_df1.shape

(18442, 8)

In [5]:
master_data_df.columns

Index(['Postcode', 'House', 'Semi-detached', 'Apartment', 'Other dwelling',
       'No bedrooms (studios, etc)', '1 bedroom', '2 bedrooms', '3 bedrooms',
       '4 bedrooms', '5 bedrooms', '6 or more bedrooms',
       'Couple with no children', 'Couple with children',
       'One parent with children', 'Other family', 'Sole person household',
       'Group household', 'People with Mental Health Conditions',
       'Total People', 'State'],
      dtype='object')

In [6]:
lat_lng_df1.columns

Index(['id', 'Postcodes', 'locality', 'state', 'lat', 'long', 'Lat_precise',
       'Long_precise'],
      dtype='object')

In [7]:
master_data_df.dtypes

Postcode                                 int64
House                                    int64
Semi-detached                            int64
Apartment                                int64
Other dwelling                           int64
No bedrooms (studios, etc)               int64
1 bedroom                                int64
2 bedrooms                               int64
3 bedrooms                               int64
4 bedrooms                               int64
5 bedrooms                               int64
6 or more bedrooms                       int64
Couple with no children                  int64
Couple with children                     int64
One parent with children                 int64
Other family                             int64
Sole person household                    int64
Group household                          int64
People with Mental Health Conditions     int64
Total People                             int64
State                                   object
dtype: object

In [8]:
lat_lng_df1.dtypes

id                int64
Postcodes         int64
locality         object
state            object
lat             float64
long            float64
Lat_precise     float64
Long_precise    float64
dtype: object

In [ ]:
lat_lng_df1 = lat_lng_df1.drop_duplicates(subset=["Postcodes"], keep='first')

In [ ]:
lat_lng_df1.shape

In [ ]:
master_data_with_coords_df = pd.merge(master_data_df, lat_lng_df1, how="left", left_on=["Postcode"], right_on=["Postcodes"])
master_data_with_coords_df = master_data_with_coords_df.drop("Postcodes",axis=1)
master_data_with_coords_df.head()                                           

In [ ]:
master_data_with_coords_df = master_data_with_coords_df.drop('state', axis=1)
master_data_with_coords_df = master_data_with_coords_df.rename(columns={"State_x": "State"})

In [ ]:
master_data_with_coords_df.shape

In [ ]:
master_data_with_coords_df.head()

In [ ]:
pop_greater_10000_coord_df = master_data_with_coords_df.loc[master_data_with_coords_df["Total People"] >= 10000]

In [ ]:
pop_greater_10000_coord_df.shape

In [ ]:
wa_pop_greater_10000_df = pop_greater_10000_coord_df.loc[pop_greater_10000_coord_df["State"] == "5: Western Australia"]

In [ ]:
wa_pop_greater_10000_df.shape

In [ ]:
wa_pop_greater_10000_df["Mental Health Ratio"]=wa_pop_greater_10000_df["People with Mental Health Conditions"] / wa_pop_greater_10000_df["Total People"]*100

In [ ]:
wa_pop_greater_10000_df.tail(20)

In [ ]:
# Store WA Postcode by WA greater than 10,000 population 
wa_pg10000_postcode = wa_pop_greater_10000_df["Postcode"].astype(int)


In [ ]:
wa_pg10000_postcode

In [ ]:
wa_pg10000_locations1 = wa_pop_greater_10000_df[["lat", "long"]].astype(float)

In [ ]:
wa_pg10000_locations1

In [ ]:
#wa_pg10000_locations = wa_pop_greater_10000_df[["Latitude", "Longitude"]].astype(float)

In [ ]:
#wa_pg10000_locations

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [ ]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

In [ ]:
# Convert Mental Health to float and store
# HINT: be sure to handle NaN values

loc_mental_health = wa_pop_greater_10000_df["Mental Health Ratio"].astype(float)


In [ ]:

#Heatmap

#marker_layer = gmaps.symbol_layer(wa_pg10000_locations)
fig_mental_health = gmaps.figure(center = (-28, 132) ,zoom_level = 3.7)
mental_health_layer = gmaps.heatmap_layer(wa_pg10000_locations1, weights=loc_mental_health, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 0.5)
fig_mental_health.add_layer(mental_health_layer)
#fig_mental_health.add_layer(marker_layer)
fig_mental_health


In [ ]:
# set up additional columns to hold information
wa_pop_greater_10000_df['Gyms'] = ""
gyms_df = wa_pop_greater_10000_df.reset_index(drop=True)
gyms_df.head(20)

In [ ]:
# Testing Autocomplete

# Set the hotel search radius

search_type = "gym"
search_radius = 10000

search_loc = "" 

count = 0

params = {
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
    "postal_code":  
}

# use iterrows to iterate through pandas dataframe
for index, row in wa_pop_greater_10000_df.iterrows():
    # get location from df
    search_postcode = row['Postcode']
 
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    count=count+1
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    params["location"] = search_loc
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    search_result = requests.get(base_url, params=params)
    gym_name = search_result.json()
    
    #Store the first hotel in dataframe
    
    try:
    # Print the name and address of the first hotel to appear
        print(f"{count} - A Gym in {search_postcode} to try is {gym_name['results'][0]['name']}.")
        wa_pop_greater_10000_df.loc[index, "Gym"] = gym_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        wa_pop_greater_10000_df.loc[index, "Gym"] = "N/A"
        print("No results... skipping.")
        
    #print("----------------------------")

In [ ]:
# Set the hotel search radius

search_type = "gym"
search_radius = 10000

search_loc = "" 

count = 0

params = {
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in wa_pop_greater_10000_df.iterrows():
    # get location from df
    search_postcode = row['Postcode']
 
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    count=count+1
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    params["location"] = search_loc
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    search_result = requests.get(base_url, params=params)
    gym_name = search_result.json()
    
    #Store the first hotel in dataframe
    
    try:
    # Print the name and address of the first hotel to appear
        print(f"{count} - A Gym in {search_postcode} to try is {gym_name['results'][0]['name']}.")
        wa_pop_greater_10000_df.loc[index, "Gym"] = gym_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        wa_pop_greater_10000_df.loc[index, "Gym"] = "N/A"
        print("No results... skipping.")
        
    #print("----------------------------")

In [ ]:
gym_name["results"]